<a href="https://colab.research.google.com/github/Benmoussa-marouane/camping/blob/master/camping_V2_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### import libraries

In [1]:
# Obviously
import pandas as pd


# !pip install xgboost
import numpy as np 

# To build our pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# !pip install impyute
# from impyute.imputation.cs import mice

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

# Other helpers
from sklearn.metrics import accuracy_score, classification_report

from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


!pip install category_encoders

from category_encoders.target_encoder import TargetEncoder

1.14.0
     |████████████████████████████████| 92kB 8.8MB/s 


In [2]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow
print(tensorflow.__version__)

1.14.0


Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Data scientist')

!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
drive  sample_data


### The Dataset

In [4]:
communes_df=pd.read_pickle('/content/drive/My Drive/Colab Notebooks/Data scientist/data/commune_scoring.pkl')
# Separation Variables d'intérêts et variables explicatives
print(communes_df.camping.value_counts())
print(communes_df.info())

0    31610
1     5067
Name: camping, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36677 entries, 0 to 36676
Data columns (total 96 columns):
codgeo                                                             36677 non-null object
nb_pharmacies_et_parfumerie                                        36676 non-null float64
dynamique_entrepreneuriale                                         36676 non-null float64
dynamique_entrepreneuriale_service_et_commerce                     36676 non-null float64
synergie_medicale_commune                                          36677 non-null int64
orientation_economique                                             36677 non-null object
indice_fiscal_partiel                                              36677 non-null float64
score_fiscal                                                       36677 non-null float64
indice_synergie_medicale                                           36677 non-null float64
score_synergie_medicale            

### keep in mind data in unbalanced we need to pick carefully good evaluation metric, maybe even oversampling..


In [5]:
dff = communes_df.select_dtypes(exclude=["number","bool"])
df_categorical = dff.assign(score_equipement_de_sante_bv=communes_df['score_equipement_de_sante_bv'])
df_categorical = df_categorical.fillna(df_categorical.mode().iloc[0]) # impute missing data using Mode or most frequent For categorical data 
# df_categorical = df_categorical.str.strip()

string_features = ['orientation_economique','seg_croissance_pop','libgeo','urbanite_ruralite','dynamique_demographique_bv',
                   'seg_environnement_demographique_obsolete','environnement_demographique'
                  ,'syn_medical','seg_cap_fiscale','seg_dyn_entre','dyn_setc']

# for fe in string_features:
#     df_categorical[fe] = df_categorical[fe].map(lambda x: "".join(w.strip() for w in x))



##############################

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
X_incomplete = communes_df.select_dtypes(include=numerics).astype(float)
df_numeric = X_incomplete[X_incomplete.columns.difference(['score_equipement_de_sante_bv'])]### score_equipment_de_sante  is categorical

print(len(X_incomplete))

36677


In [6]:
df_numeric.columns

Index(['camping', 'capacite_fisc', 'capacite_fiscale', 'capacite_hotel',
       'densite_medicale_bv', 'dep_moyenne_salaires_cadre_horaires',
       'dep_moyenne_salaires_employe_horaires',
       'dep_moyenne_salaires_horaires', 'dep_moyenne_salaires_ouvrie_horaires',
       'dep_moyenne_salaires_prof_intermediaire_horaires',
       'dynamique_demographique_insee', 'dynamique_entrepreneuriale',
       'dynamique_entrepreneuriale_service_et_commerce', 'evolution_pop_',
       'evolution_population', 'indice_demographique', 'indice_fiscal_partiel',
       'indice_menages', 'indice_synergie_medicale',
       'moyenne_revenus_fiscaux_departementaux',
       'moyenne_revenus_fiscaux_regionaux', 'moyenne_revnus_fiscaux',
       'nb_actifs_non_salaries', 'nb_actifs_salaries', 'nb_atifs',
       'nb_creation_commerces', 'nb_creation_construction',
       'nb_creation_enteprises', 'nb_creation_industrielles',
       'nb_creation_services', 'nb_de_commerce',
       'nb_de_services_aux_particuli

In [0]:
for f in df_numeric.columns:
  X_incomplete[f].fillna((X_incomplete[f].mean()), inplace=True)

In [8]:
![categorical_dist](https://drive.google.com/file/d/1CKOwndhp47xm0sAxLe8EkYaiyhAe77zL/view?usp=sharing)

/bin/bash: -c: line 0: syntax error near unexpected token `https://drive.google.com/file/d/1CKOwndhp47xm0sAxLe8EkYaiyhAe77zL/view?usp=sharing'
/bin/bash: -c: line 0: `[categorical_dist](https://drive.google.com/file/d/1CKOwndhp47xm0sAxLe8EkYaiyhAe77zL/view?usp=sharing)'


In [0]:
# Source: https://maxhalford.github.io/blog/target-encoding-done-the-right-way/
def calc_smooth_mean(df1, cat_name, target, weight):
    # Compute the global mean
    mean = communes_df[target].mean()

    # Compute the number of values and the mean of each group
    agg = communes_df.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    return df1[cat_name].map(smooth)

**Feature** 'codgeo' has 36677 unique categories note good idea to use target encoding

Feature 'libgeo' has 34125 unique categories

Feature 'dep' has 100 unique categories

Feature 'cp' has 100 unique categories

In [0]:
WEIGHT = 300
df_categorical['codgeo'] = calc_smooth_mean(df1=communes_df, cat_name='codgeo', target='camping', weight=WEIGHT).astype('category')
df_categorical['libgeo'] = calc_smooth_mean(df1=communes_df, cat_name='libgeo', target='camping', weight=WEIGHT).astype('category')
df_categorical['dep'] = calc_smooth_mean(df1=communes_df, cat_name='dep', target='camping', weight=WEIGHT).astype('category')
df_categorical['cp'] = calc_smooth_mean(df1=communes_df, cat_name='cp', target='camping', weight=WEIGHT).astype('category')

In [11]:
#0.137693	0.137693	0.0883396	0.0883396
#
#

df_categorical[['codgeo','libgeo','dep','cp']].head()

,codgeo,libgeo,dep,cp
0,0.137693,0.137693,0.143874,0.143874
1,0.137693,0.137693,0.143874,0.143874
2,0.137693,0.137693,0.143874,0.143874
3,0.141015,0.141015,0.143874,0.143874
4,0.137693,0.137693,0.143874,0.143874


In [12]:
# Decide which categorical variables you want to use in model
for col_name in df_categorical.columns:
#     if df_numeric[col_name].dtypes == 'object':
      unique_cat = len(df_categorical[col_name].unique())
      print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'codgeo' has 2 unique categories
Feature 'orientation_economique' has 5 unique categories
Feature 'seg_croissance_pop' has 2 unique categories
Feature 'libgeo' has 35 unique categories
Feature 'dep' has 99 unique categories
Feature 'urbanite_ruralite' has 6 unique categories
Feature 'dynamique_demographique_bv' has 7 unique categories
Feature 'seg_environnement_demographique_obsolete' has 8 unique categories
Feature 'environnement_demographique' has 7 unique categories
Feature 'syn_medical' has 3 unique categories
Feature 'seg_cap_fiscale' has 3 unique categories
Feature 'seg_dyn_entre' has 3 unique categories
Feature 'dyn_setc' has 3 unique categories
Feature 'cp' has 99 unique categories
Feature 'score_equipement_de_sante_bv' has 5 unique categories


In [13]:
df_categorical['urbanite_ruralite'].value_counts()

Com rurale < 2 000 m habts    30052
Com < 10 m habts               2505
Com > 200 m habts              1369
Com < 50 m habts               1325
Com < 200 m habts               836
Com rurale > 2 000 habts        590
Name: urbanite_ruralite, dtype: int64

In [14]:
# In this case, bucket low frequecy categories as "Other"
df_categorical['urbanite_ruralite'] = ['Com rurale < 2 000 m habts ' if x == 'Com rurale < 2 000 m habts' else 'Other' for x in df_categorical['urbanite_ruralite']]

print(df_categorical['urbanite_ruralite'].value_counts().sort_values(ascending=False))

Com rurale < 2 000 m habts     30052
Other                           6625
Name: urbanite_ruralite, dtype: int64


In [15]:
df_categorical['seg_cap_fiscale'].value_counts() 

Fiscalité moyenne    28780
Fiscalité faible      6628
Fiscalité élevée      1269
Name: seg_cap_fiscale, dtype: int64

In [16]:
# In this case, bucket low frequecy categories as "Other"
df_categorical['seg_cap_fiscale'] = ['Fiscalité moyenne' if x == 'Fiscalité moyenne' else 'Other' for x in df_categorical['seg_cap_fiscale']]

print(df_categorical['seg_cap_fiscale'].value_counts().sort_values(ascending=False))

Fiscalité moyenne    28780
Other                 7897
Name: seg_cap_fiscale, dtype: int64


In [17]:
df_categorical['syn_medical'].value_counts()

Faible Synergie Médicale    24669
Forte Synergie Médicale      6048
Synergie Médicale            5960
Name: syn_medical, dtype: int64

In [18]:
# In this case, bucket low frequecy categories as "Other"
df_categorical['syn_medical'] = ['Faible Synergie Médicale' if x == 'Faible Synergie Médicale' else 'Other' for x in df_categorical['syn_medical']]

print(df_categorical['syn_medical'].value_counts().sort_values(ascending=False))

Faible Synergie Médicale    24669
Other                       12008
Name: syn_medical, dtype: int64


In [19]:
df_categorical['seg_dyn_entre'].value_counts()

Faible dynamique        30187
Dynamique Economique     5059
Moyenne dynamique        1431
Name: seg_dyn_entre, dtype: int64

In [20]:
# In this case, bucket low frequecy categories as "Other"
df_categorical['seg_dyn_entre'] = ['Faible dynamique' if x == 'Faible dynamique' else 'Other' for x in df_categorical['seg_dyn_entre']]

print(df_categorical['seg_dyn_entre'].value_counts().sort_values(ascending=False))

Faible dynamique    30187
Other                6490
Name: seg_dyn_entre, dtype: int64


In [21]:
df_categorical.dyn_setc.value_counts()

Faible Dynamique Serv et Com              31916
Bonne Dynamique Entreprise Serv et Com     3788
Dynamique Serv et Com                       973
Name: dyn_setc, dtype: int64

In [22]:
# In this case, bucket low frequecy categories as "Other"
df_categorical['dyn_setc'] = ['Faible Dynamique Serv et Com' if x == 'Faible Dynamique Serv et Com' else 'Other' for x in df_categorical['dyn_setc']]

print(df_categorical['dyn_setc'].value_counts().sort_values(ascending=False))

Faible Dynamique Serv et Com    31916
Other                            4761
Name: dyn_setc, dtype: int64


In [0]:
# df_categorical.seg_environnement_demographique_obsolete.value_counts()

In [0]:
# # In this case, bucket low frequecy categories as "Other"
# for x in df_categorical['seg_environnement_demographique_obsolete'] :
  
#     if x is 'Zone rurale en croissance démographique':  
#         df_categorical['seg_environnement_demographique_obsolete'] = 'Zone rurale en croissance démographique'
    
#     elif x is 'Zone rurale en déclin démographique':  
#         df_categorical['seg_environnement_demographique_obsolete'] = 'Zone rurale en déclin démographique'
    
#     else:
#         df_categorical['seg_environnement_demographique_obsolete'] = 'other'
        
        
    
# print(df_categorical['seg_environnement_demographique_obsolete'].value_counts().sort_values(ascending=False))

In [25]:
for c in df_categorical.columns:
  print(df_categorical[c].value_counts().sort_values(ascending=False).head(10))
  print()
 

0.137693    31610
0.141015     5067
Name: codgeo, dtype: int64

Bassin Résidentiel         16870
Bassin Urbain               8671
Bassin Industriel           7122
Bassins Agroalimentaire     2564
Bassin diversifié           1450
Name: orientation_economique, dtype: int64

en croissance démographique    36666
en déclin démographique           11
Name: seg_croissance_pop, dtype: int64

0.137693    27899
0.141015     4613
0.137237     1824
0.136784      558
0.140548      446
0.136334      244
0.140084      162
0.135887      150
0.139624      116
0.135443       66
Name: libgeo, dtype: int64

0.149452    894
0.082463    866
0.063123    816
0.107621    782
0.095163    745
0.072277    730
0.082866    707
0.104817    706
0.064900    693
0.076354    675
Name: dep, dtype: int64

Com rurale < 2 000 m habts     30052
Other                           6625
Name: urbanite_ruralite, dtype: int64

Grande Ville                                          8671
1.Accroissement par excédent naturel et migratoi

##pipeline


We'll define a new `ColumnTransformer` object that keeps our numerical features and apply one hot encoding on our categorical features. 

That will allow us to create a clean pipeline that includes both features engineering (one hot encoding here) and training the model (a nice way to avoid data leakage)

In [0]:
numeric_features =list(df_numeric)
numeric_features.remove('camping')

categorical_features =list(df_categorical)

te_features = ['codgeo','libgeo','dep','cp']


for i in te_features:
  categorical_features.remove(i)
 


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])


                                 
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
         ])

                                 
# target_encoding = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('TE', ce.TargetEncoder(cols=te_features) )
#          ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])



In [27]:
# print(data.shape)

df_numeric = pd.DataFrame(data=X_incomplete,index= range(0,len(X_incomplete)),columns=X_incomplete.columns)  
# print(df_numeric.shape)

merged =pd.concat([df_numeric, df_categorical], axis=1)


#duplicated columns

merged = merged.loc[:,~merged.columns.duplicated()]

print(merged.shape)

(36677, 96)


Now we can define our 4 models as sklearn `Pipeline` objects, containing our preprocessing step and training of one given algorithm.

Let's split the data into training and test sets.

In [28]:
# Get X, y
y = merged['camping']
X = merged.drop("camping", axis=1)

# X.reset_index(drop=True).head()
np.shape(X)

(36677, 95)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=.3)

#MLP using keras


## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model—in this example, the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. We ask the model to make predictions about a test set—in this example, the `test_images` array. We verify that the predictions match the labels from the `test_labels` array.

To start training,  call the `model.fit` method—the model is "fit" to the training data:

In [0]:
def myModel():
    model = keras.Sequential([
        keras.layers.Dense(32, input_shape=(123,),activation = 'relu'),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    
    return model


In [57]:
from keras.wrappers.scikit_learn import KerasClassifier

from keras import callbacks


early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

# wrap the model using the function you created
clf = KerasClassifier(build_fn=myModel, epochs=50, batch_size=32, verbose=1,validation_split=0.2, callbacks=[early_stopping])

# Now we have a full prediction pipeline.
Mdl = Pipeline(steps=[('preprocessor', preprocessor), ('clf',clf)])



Mdl.fit(X_train, y_train)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 32)                3968      
_________________________________________________________________
dense_44 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 129       
Total params: 8,321
Trainable params: 8,321
Non-trainable params: 0
_________________________________________________________________
Train on 20538 samples, validate on 5135 samples
Epoch 1/50
20538/20538 [==============================] - 2s 115us/step - loss: 0.3234 - acc: 0.8766 - val_loss: 0.2944 - val_acc: 0.8896
Epoch 2/50
20538/20538 [==============================] - 1s 62us/step - loss: 0.2871 - acc: 0.8911 - val_loss: 0.2797 - val_acc: 0.8931
Epoch 3/50
20538/20538 [==============================] -

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [0]:
history=Mdl.named_steps["clf"]..history

import matplotlib.pyplot as plt
def plot_training(history):
    # Plot training & validation accuracy values
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    plt.savefig('acc_vs_epochs.png')
    
plot_training(history)

## Make predictions

With the model trained, we can use it to make predictions about some images.

In [59]:
predictions = Mdl.predict(X_test)
predictions

11004/11004 [==============================] - 0s 15us/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [60]:
predictions[0]

array([0.])

A prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different articles of clothing. We can see which label has the highest confidence value:

In [61]:
np.argmax(predictions[0])

0

So the model is most confident that sample 0 i test data is 0

In [62]:
y_test[0]

0.0